# Halloween Candy Basket

<img src="candy_basket.jpeg" style="max-width:200px;margin-left: 0; margin-right: auto;"/>

Goal: create a basket of 50 candies that minimizes cost and has

* an average deliciousness of at least 50%
* no more than 6% average fat content per piece of candy
* no more than 1.2% average salt per piece
* no more than 13% average sugar per piece
* contains at least some of each candy

In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
from pulp import *

In [2]:
candy = pd.read_csv('candy.csv')

In [3]:
candy

,Candy,Fat,Salt,Sugar,Deliciousness,Price
0,Reeses,0.20,0.02,0.15,0.82,0.64
1,Hersheys,0.09,0.01,0.09,0.64,0.55
2,Milky Way,0.07,0.02,0.11,0.71,0.61
3,3 Musketeers,0.05,0.02,0.18,0.50,0.45
4,Twix,0.20,0.03,0.16,0.90,0.69
5,Skittles,0.01,0.00,0.30,0.30,0.15
6,Starburst,0.03,0.00,0.20,0.34,0.21
7,Hubba Bubba,0.00,0.00,0.10,0.04,0.05


### Problem set up

In [4]:
model = LpProblem("Halloween Basket Problem", LpMinimize)

First we define our variables. Each piece of candy is an integer.

The basket has to contain at least 1 of each candy.

In [5]:
reeses = LpVariable("Reeses", lowBound=1, cat=LpInteger)
hersheys = LpVariable("Hersheys", lowBound=1, cat=LpInteger)
milkyway = LpVariable("Milky_Way", lowBound=1, cat=LpInteger)
musketeer = LpVariable("3_Musketeers", lowBound=1, cat=LpInteger)
twix = LpVariable("Twix", lowBound=1, cat=LpInteger)
skittles = LpVariable("Skittles", lowBound=1, cat=LpInteger)
starburst = LpVariable("Starburst", lowBound=1, cat=LpInteger)
hubbabubba = LpVariable("Hubba_Bubba", lowBound=1, cat=LpInteger)

variables = [reeses, hersheys, milkyway, musketeer, twix, skittles, starburst, hubbabubba]

### Objective Function
Here we define what we are trying to minimize. In this case it's the cost of our Halloween basket.

In [6]:
objective_function = []

for var in variables:
    var_name = var.name.replace('_',' ')
    price = candy[candy['Candy']==var_name].Price.iloc[0]
    objective_function.append(price * var)

model += lpSum(objective_function), "Total Cost of Candies"

We can inspect our model and see that the we now have a minimization objective and a set of candy variables, each of which must be at least 1.

In [7]:
model

Halloween_Basket_Problem:
MINIMIZE
0.45*3_Musketeers + 0.55*Hersheys + 0.05*Hubba_Bubba + 0.61*Milky_Way + 0.64*Reeses + 0.15*Skittles + 0.21*Starburst + 0.69*Twix + 0.0
VARIABLES
1 <= 3_Musketeers Integer
1 <= Hersheys Integer
1 <= Hubba_Bubba Integer
1 <= Milky_Way Integer
1 <= Reeses Integer
1 <= Skittles Integer
1 <= Starburst Integer
1 <= Twix Integer

### Adding constraints
Now we add the various constraints around delciousness, nutritional content, and the size of the Halloween basket.

In [8]:
fat = []
for ix, f in enumerate(candy.Fat):
    fat.append(f * variables[ix])

salt = []
for ix, s in enumerate(candy.Salt):
    salt.append(s * variables[ix])    
    
sugar = []
for ix, su in enumerate(candy.Sugar):
    sugar.append(su * variables[ix])        
    
deliciousness = []
for ix, d in enumerate(candy.Deliciousness):
    deliciousness.append(d * variables[ix])

In [9]:
model += lpSum(deliciousness)/50 >= 0.5, "Deliciousness Constraint"
model += lpSum(fat)/50 <= 0.06, "Fat Constraint"
model += lpSum(salt)/50 <= 0.012, "Salt Constraint"
model += lpSum(sugar)/50 <= 0.13, "Sugar Constraint"

model += lpSum(variables) == 50, "At least 50 candies"

We can take a look at our model to make sure that the constraints look correct. 

> Note: don't forget that our main constraints are weighted averages, that's why coefficients are small (they are the nutritional values divided by 50)!

In [10]:
model

Halloween_Basket_Problem:
MINIMIZE
0.45*3_Musketeers + 0.55*Hersheys + 0.05*Hubba_Bubba + 0.61*Milky_Way + 0.64*Reeses + 0.15*Skittles + 0.21*Starburst + 0.69*Twix + 0.0
SUBJECT TO
Deliciousness_Constraint: 0.01 3_Musketeers + 0.0128 Hersheys
 + 0.0008 Hubba_Bubba + 0.0142 Milky_Way + 0.0164 Reeses + 0.006 Skittles
 + 0.0068 Starburst + 0.018 Twix >= 0.5

Fat_Constraint: 0.001 3_Musketeers + 0.0018 Hersheys + 0.0014 Milky_Way
 + 0.004 Reeses + 0.0002 Skittles + 0.0006 Starburst + 0.004 Twix <= 0.06

Salt_Constraint: 0.0004 3_Musketeers + 0.0002 Hersheys + 0.0004 Milky_Way
 + 0.0004 Reeses + 0.0006 Twix <= 0.012

Sugar_Constraint: 0.0036 3_Musketeers + 0.0018 Hersheys + 0.002 Hubba_Bubba
 + 0.0022 Milky_Way + 0.003 Reeses + 0.006 Skittles + 0.004 Starburst
 + 0.0032 Twix <= 0.13

At_least_50_candies: 3_Musketeers + Hersheys + Hubba_Bubba + Milky_Way
 + Reeses + Skittles + Starburst + Twix = 50

VARIABLES
1 <= 3_Musketeers Integer
1 <= Hersheys Integer
1 <= Hubba_Bubba Integer
1 <= Milky

### Running the model and inspecting the results

In [11]:
model.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/brandonrose/opt/anaconda3/envs/b39/lib/python3.9/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/pp/vgfp1wf143q46m_8v6qbt3bw0000gn/T/2688bd15ea57405e954183341a19129a-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/pp/vgfp1wf143q46m_8v6qbt3bw0000gn/T/2688bd15ea57405e954183341a19129a-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 10 COLUMNS
At line 71 RHS
At line 77 BOUNDS
At line 86 ENDATA
Problem MODEL has 5 rows, 8 columns and 36 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 20.4348 - 0.00 seconds
Cgl0004I processed model has 5 rows, 8 columns (8 integer (0 of which binary)) and 36 elements
Cutoff increment increased from 1e-05 to 0.00999
Cbc0012I Integer solution of 20.55 found by DiveCoefficient after 59 iterations and 0 nodes (0.01 s

1

In [12]:
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")
print('\n')

for var in model.variables():
    print(f"{var.name.replace('_',' ')}: {var.value()}")

status: 1, Optimal
objective: 20.55


3 Musketeers: 1.0
Hersheys: 15.0
Hubba Bubba: 8.0
Milky Way: 13.0
Reeses: 1.0
Skittles: 2.0
Starburst: 9.0
Twix: 1.0


### Constraint checking

This can be done automatically but we can do it manually for a sanity check if we'd like. To print them automatically use:

```
for name, constraint in model.constraints.items():
    print(f"{name}: {constraint.value()}")
```

In [13]:
basket = 0
for var in model.variables():
    basket += var.value()
print(f"Total size of candy basket: {basket}")

nutritions = ['Fat','Salt','Sugar','Deliciousness']

for x in nutritions:
    nutrition = 0
    for var in model.variables():
        nutrition += candy[candy['Candy']==var.name.replace('_',' ')][x].iloc[0] * var.value()
    print(f"{x}: {round(nutrition/50,2)}")

Total size of candy basket: 50.0
Fat: 0.06
Salt: 0.01
Sugar: 0.13
Deliciousness: 0.5
